In [1]:
import pandas as pd
path="D:/labs/ML/Learning/finals/datasets/dataDecisionTree.csv"
data=pd.read_csv(path)
data

,Outlook,Temp,Humidity,Wind,Decision
0,Sunny,85,85,Weak,No
1,Sunny,80,90,Strong,No
2,Overcast,83,78,Weak,Yes
3,Rain,70,96,Weak,Yes
4,Rain,68,80,Weak,Yes
5,Rain,65,70,Strong,No
6,Overcast,64,65,Strong,Yes
7,Sunny,72,95,Weak,No
8,Sunny,69,70,Weak,Yes
9,Rain,75,80,Weak,Yes


In [2]:
n=len(data)
for i in range(n):
    if data.iat[i,1]>=80: data.iat[i,1]='hot'
    elif data.iat[i,1]>=70: data.iat[i,1]='mild'
    else: data.iat[i,1]='cool'
    if data.iat[i,2]>=80: data.iat[i,2]='high'
    else : data.iat[i,2]='normal'   
print(data)

     Outlook  Temp Humidity    Wind Decision
0      Sunny   hot     high    Weak       No
1      Sunny   hot     high  Strong       No
2   Overcast   hot   normal    Weak      Yes
3       Rain  mild     high    Weak      Yes
4       Rain  cool     high    Weak      Yes
5       Rain  cool   normal  Strong       No
6   Overcast  cool   normal  Strong      Yes
7      Sunny  mild     high    Weak       No
8      Sunny  cool   normal    Weak      Yes
9       Rain  mild     high    Weak      Yes
10     Sunny  mild   normal  Strong      Yes
11  Overcast  mild     high  Strong      Yes
12  Overcast   hot   normal    Weak      Yes
13      Rain  mild     high  Strong       No


In [3]:
import math
def entropy(attribute,target):
    categories=list(set(attribute))
    frequency={}
    entropies={}
    # entropies=[]
    for cat in categories:
        frequency[cat]=attribute.count(cat)
    for cat in categories:
        yes,no,ent=0,0,0
        for a,t in zip(attribute,target):
            if a==cat:
                if t=='Yes': yes+=1
                else: no+=1
        py=yes/frequency[cat]
        pn=no/frequency[cat]
        if py==0 or pn==0: ent=0
        else: ent=-(py*math.log2(py))-(pn*math.log2(pn))
        entropies[cat]=ent
        # entropies.append(ent)
    return entropies       
def categoryProbabilities(attribute): 
    categories=list(set(attribute))
    probabilities={}
    # probabilities=[]    
    for cat in categories:
        freq=attribute.count(cat)
        probabilities[cat]=freq/n
        # probabilities.append(freq/n)
    return probabilities
def infoGain(entropies,probabs,decisionEntropy):
    sum=0
    for e,p in zip(entropies.values(),probabs.values()):
        sum+=(e*p)
    ig=decisionEntropy-sum
    return ig
def calcSplitInfo(probabilities):
    val=0
    for probab in probabilities:
        val-=probab*math.log2(probab)
    return val
def gainRatio(s,g):
    return (g/s)
def getClass(attribute,category,target):
    yes,no=0,0
    for a,t in zip(attribute,target):
        if a==category:
            # print(a,t)
            if t=='Yes': yes+=1
            if t=='No': no+=1
    if yes>no: return 'Yes'
    else: return 'No'


In [4]:
yes,no=0,0
for d in data['Decision']:
    if d=='No': no+=1
    else: yes+=1
pn=no/n
py=yes/n
decisionEntropy=-(pn*math.log2(pn))-(py*math.log2(py))
print(decisionEntropy)  

0.9402859586706311


In [5]:
def getRoot(data):    
    attr=list(data.columns)
    attr.remove('Decision')
    attributeEntropies={}
    attributeProbabs=[]
    informationGains={}
    splitInformation={}
    gainRatios={}
    for a in attr:
        entr=entropy(list(data[a]),list(data['Decision']))
        prob=categoryProbabilities(list(data[a]))
        attributeEntropies[a]=entr
        attributeProbabs.append(prob)
    # print(attributeProbabs)
    for a,p in zip(attr,attributeProbabs):
        si=calcSplitInfo(p.values())
        splitInformation[a]=si
    for e,p,a in zip(attributeEntropies.values(),attributeProbabs,attr):
        gain=infoGain(e,p,decisionEntropy)
        informationGains[a]=gain
    for g,s,a in zip(informationGains.values(),splitInformation.values(),attr):
        gr=gainRatio(s,g)
        gainRatios[a]=gr
    # print("information gains")
    # print(informationGains)
    print(splitInformation)
    m=max(list(gainRatios.values()))
    for k,v in gainRatios.items():
        if v==m:
            root=k
            break
    print(f"root is: {root}")
    for k,v in attributeEntropies[root].items():
        # print(k,v)
        if v==0:
            # print(k,v)
            classified=getClass(data[root],k,data['Decision'])
            print(f"{k}-->{classified}")
            break
    return root

In [6]:
finalTree=''
root=getRoot(data)

{'Outlook': 1.5774062828523454, 'Temp': 1.5566567074628228, 'Humidity': 0.9852281360342516, 'Wind': 0.9852281360342516}
root is: Outlook
Overcast-->Yes


In [7]:
finalTree+=f"root is: Outlook Overcast-->Yes"

In [8]:
leftSubDataFrame=data[data['Outlook']=='Sunny']
rightSubDataFrame=data[data['Outlook']=='Rain']


In [9]:
leftSubDataFrame

,Outlook,Temp,Humidity,Wind,Decision
0,Sunny,hot,high,Weak,No
1,Sunny,hot,high,Strong,No
7,Sunny,mild,high,Weak,No
8,Sunny,cool,normal,Weak,Yes
10,Sunny,mild,normal,Strong,Yes


In [10]:
root=getRoot(leftSubDataFrame)

{'Outlook': 0.5305095811322292, 'Temp': 1.074055329306287, 'Humidity': 0.8772776505803251, 'Wind': 0.8772776505803251}
root is: Outlook


In [11]:
finalTree+=f" sunny-->root is: Humidity high-->No"

In [12]:
root=getRoot(rightSubDataFrame)

{'Outlook': 0.5305095811322292, 'Temp': 0.8772776505803251, 'Humidity': 0.7883410435920014, 'Wind': 0.8772776505803251}
root is: Outlook


In [13]:
finalTree+=f" rain-->root is: Wind weak-->Yes"

In [14]:
finalTree

'root is: Outlook Overcast-->Yes sunny-->root is: Humidity high-->No rain-->root is: Wind weak-->Yes'